# FEATURE EXTRACTION WITH DATA AUGMENTATION and Fine-tuning
1 Add your custom network on top of an already-trained base network.

2 Freeze the base network.

3 Train the part you added.

4 Unfreeze some layers in the base network.

5 Jointly train both these layers and the part you added.

You already completed the first three steps when doing feature extraction. Let’s proceed with step 4: you’ll unfreeze your conv_base and then freeze individual layers
inside it.

# Description of VGG16 model
VGG16 (also called OxfordNet) is a convolutional neural network architecture named after the Visual Geometry Group from Oxford, who developed it.
It was used to win the ILSVR (ImageNet) competition in 2014.
To this day is it still considered to be an excellent vision model, although it has been somewhat outperformed by more revent advances such as Inception and ResNet.


In [1]:
import os

In [2]:
from tensorflow.keras.applications import VGG16
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

In [3]:
# 1 Add your custom network on top of an already-trained base network.

from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2097408   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [4]:
original_dataset_dir = "train"
base_dir = 'cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
print(train_dir)
print(validation_dir)
print(test_dir)

cats_and_dogs_small\train
cats_and_dogs_small\validation
cats_and_dogs_small\test


# 2.freezing the convolutional base
In Keras, you freeze a network by setting its trainable attribute to False:

In [5]:
print('This is the number of trainable weights ''before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30


In [6]:
conv_base.trainable = False
print('This is the number of trainable weights ''after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


# Training the model end to end with a frozen convolutional base

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                rescale=1./255,
                rotation_range=40,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                horizontal_flip=True,
                fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_generator = train_datagen.flow_from_directory(
                    train_dir,
                    target_size=(150, 150),
                    batch_size=20,
                    class_mode='binary')

Found 2000 images belonging to 2 classes.


In [9]:
validation_generator = test_datagen.flow_from_directory(
                        validation_dir,
                        target_size=(150, 150),
                        batch_size=20,
                        class_mode='binary')

Found 1000 images belonging to 2 classes.


In [10]:
from tensorflow.keras import optimizers
model.compile(loss='binary_crossentropy',
                optimizer=optimizers.RMSprop(lr=2e-5),
                metrics=['acc'])

# 3.Train the part you added.

In [11]:
history = model.fit_generator(
                train_generator,
                steps_per_epoch=2, #100
                epochs=30,
                validation_data=validation_generator,
                validation_steps=50)

Epoch 1/30
2/2 [==============================] - 23s 11s/step - loss: 0.7561 - acc: 0.4250 - val_loss: 0.7179 - val_acc: 0.4670
Epoch 2/30
2/2 [==============================] - 15s 8s/step - loss: 0.6865 - acc: 0.5250 - val_loss: 0.6997 - val_acc: 0.5100
Epoch 3/30
2/2 [==============================] - 15s 8s/step - loss: 0.6965 - acc: 0.5250 - val_loss: 0.6853 - val_acc: 0.5480
Epoch 4/30
2/2 [==============================] - 15s 8s/step - loss: 0.6359 - acc: 0.6250 - val_loss: 0.6820 - val_acc: 0.5480
Epoch 5/30
2/2 [==============================] - 15s 8s/step - loss: 0.6571 - acc: 0.6000 - val_loss: 0.6677 - val_acc: 0.5780
Epoch 6/30
2/2 [==============================] - 15s 8s/step - loss: 0.6724 - acc: 0.5500 - val_loss: 0.6799 - val_acc: 0.5370
Epoch 7/30
2/2 [==============================] - 15s 8s/step - loss: 0.6608 - acc: 0.6250 - val_loss: 0.6687 - val_acc: 0.5570
Epoch 8/30
2/2 [==============================] - 15s 8s/step - loss: 0.6712 - acc: 0.5000 - val_loss: 

In [12]:
print('This is the number of trainable weights ''after freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights after freezing the conv base: 4


In [13]:
#conv_base.trainable = True

In [14]:
#print('This is the number of trainable weights ''after setting conv base to trainable:', len(model.trainable_weights))

# 4.Unfreeze some layers in the base network.
Freezing all layers up to a specific one

In [15]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [16]:
print('This is the number of trainable weights ''after Un-freezing some layers of conv base:', len(model.trainable_weights))

This is the number of trainable weights after Un-freezing some layers of conv base: 4


In [17]:
model.compile(loss='binary_crossentropy',
optimizer=optimizers.RMSprop(lr=1e-5),
metrics=['acc'])

In [18]:
epochs = 2 # 100
history = model.fit_generator(
                train_generator,
                steps_per_epoch=100,
                epochs=epochs,
                validation_data=validation_generator,
                validation_steps=50)

Epoch 1/2
100/100 [==============================] - 103s 1s/step - loss: 0.5358 - acc: 0.7505 - val_loss: 0.4353 - val_acc: 0.8340
Epoch 2/2
100/100 [==============================] - 102s 1s/step - loss: 0.4897 - acc: 0.7870 - val_loss: 0.3907 - val_acc: 0.8530
